In [2]:
from datasets import load_dataset

ds = load_dataset("json", data_files={'train': './data/musique_hotpot_wiki2_nq_tqa_sqd/flan_t5_xl/binary_silver/train.json'})

In [3]:
for i in ds['train']:
    print(i)
    break

print(len(ds['train']))


{'answer': 'C', 'answer_description': 'multi', 'dataset_name': 'musique', 'id': '2hop__482757_12019', 'question': 'When was the institute that owned The Collegian founded?', 'total_answer': None}
3692


In [4]:
# load /apdcephfs/private_zivtang/code/Adaptive-RAG/processed_data/musique/dev_500_subsampled.jsonl
import json
with open('/apdcephfs/private_zivtang/code/Adaptive-RAG/processed_data/musique/dev_500_subsampled.jsonl', 'r') as f:
    data = [json.loads(i) for i in f.readlines()]

In [5]:
# read /apdcephfs/private_zivtang/code/Adaptive-RAG/classifier/data/musique_hotpot_wiki2_nq_tqa_sqd/flan_t5_xl/silver/train.json
with open('/apdcephfs/private_zivtang/code/Adaptive-RAG/classifier/data/musique_hotpot_wiki2_nq_tqa_sqd/flan_t5_xl/silver/train.json', 'r') as f:
    data = json.load(f)

In [6]:

# 统计 zero one multiple 的数量
num_zero = 0
num_one = 0
num_multiple = 0
for i in data:
    if 'zero' in i['total_answer']:
        num_zero += 1
    if 'one' in i['total_answer']:
        num_one += 1
    if 'multiple' in i['total_answer']:
        num_multiple += 1
print(num_zero, num_one, num_multiple)



424 1008 1110


In [1]:
# load /apdcephfs/private_zivtang/code/Adaptive-RAG/classifier/outputs/musique_hotpot_wiki2_nq_tqa_sqd/model/t5-large/flan_t5_xl/epoch/30/2024_08_19/11_12_28/valid/dict_id_pred_results.json
import json
# pred_results = json.load(open('/apdcephfs/private_zivtang/code/Adaptive-RAG/MAB/results/2024-08-27-14-01-15_debug_valid_acc/dict_id_pred_results.json', 'r'))

train_data = json.load(open('/apdcephfs/private_zivtang/code/Adaptive-RAG/classifier/data/musique_hotpot_wiki2_nq_tqa_sqd/flan_t5_xl/silver/train.json', 'r'))
# load /apdcephfs/private_zivtang/code/Adaptive-RAG/classifier/data/musique_hotpot_wiki2_nq_tqa_sqd/flan_t5_xl/silver/valid.json
valid_data = json.load(open('/apdcephfs/private_zivtang/code/Adaptive-RAG/classifier/data/musique_hotpot_wiki2_nq_tqa_sqd/flan_t5_xl/silver/valid.json', 'r'))
predict_data = json.load(open('/apdcephfs/private_zivtang/code/Adaptive-RAG/classifier/data/musique_hotpot_wiki2_nq_tqa_sqd/predict.json', 'r'))

train_app = json.load(open('/apdcephfs/private_zivtang/code/Adaptive-RAG/classifier/data/musique_hotpot_wiki2_nq_tqa_sqd/binary/total_data_train.json', 'r'))


1350

In [7]:
# concat train_data and train_app
for i in range(len(train_app)):
    if train_app[i]['answer'] == 'B':
        train_app[i]['total_answer'] = 'one'
    elif train_app[i]['answer'] == 'C':
        train_app[i]['total_answer'] = 'multiple'

train_data = train_data + train_app
# save to /apdcephfs/private_zivtang/code/Adaptive-RAG/MAB/train_append.json
with open('/apdcephfs/private_zivtang/code/Adaptive-RAG/MAB/train_append.json', 'w') as f:
    json.dump(train_data, f)

In [4]:
# make a predict with gt

valid_data_ids = [i['id'] for i in valid_data]

for index,i in enumerate(valid_data):
    if i['id'] in valid_data_ids:
        id = valid_data_ids.index(i['id'])
        # predict_data[index]['gt_ans'] = valid_data[id]['answer']
        # predict_data[index]['gt_multi_ans'] =  valid_data[id]['total_answer']

# save the predict data with indent
with open('/apdcephfs/private_zivtang/code/Adaptive-RAG/MAB/predict.json', 'w') as f:
    json.dump(predict_data, f, indent=4)


In [2]:
num_query_per_dataset = {}
for i in predict_data:
    dataset_name = i['dataset_name']
    if dataset_name not in num_query_per_dataset:
        num_query_per_dataset[dataset_name] = 0
    num_query_per_dataset[dataset_name] += 1

# 统计每个数据集的数量
print(num_query_per_dataset)

{'musique': 500, 'hotpotqa': 500, '2wikimultihopqa': 500, 'nq': 500, 'trivia': 500, 'squad': 500}


In [5]:
# 统计answer频率

import pandas as pd
df = pd.DataFrame(valid_data)
counts = df['answer'].value_counts()

In [7]:
num_in_total_answer = 0
num_in_answer = 0
map_dict =  {'A':'zero','B':'one','C':'multiple'}
for i in valid_data:
    if i['id'] in pred_results:
        if pred_results[i['id']]['prediction'] == i['answer']:
            num_in_answer += 1
            num_in_total_answer += 1
        elif map_dict[pred_results[i['id']]['prediction']] in i['total_answer']:
            num_in_total_answer += 1
print(num_in_answer, num_in_total_answer)


789 901


In [14]:
# make a fake pred_results
pred_results = {}

for i in range(len(valid_data)):
    pred_results[valid_data[i]['id']] = {"prediction":valid_data[i]['answer'],"answer":"","dataset_name":valid_data[i]['dataset_name']}

for i in range(len(predict_data)):
    if predict_data[i]['id'] not in pred_results:
        pred_results[predict_data[i]['id']] = {"prediction": "A", "answer": "", "dataset_name": predict_data[i]['dataset_name']}
# save pred_results to json with indent
with open('oracle.json', 'w') as f:
    json.dump(pred_results, f, indent=4)

In [14]:
train_data_ids = [i['id'] for i in train_data]
valid_data_ids = [i['id'] for i in valid_data]
predict_data_ids = [i['id'] for i in predict_data]


In [15]:
# how many ids in valid_data_ids are in predict_data_ids
num = 0
for i in valid_data_ids:
    if i in predict_data_ids:
        num += 1

print(num)
print(len(valid_data_ids))

# if any train_data_ids in valid_data_ids or predict_data_ids
for i in train_data_ids:
    if i in valid_data_ids:
        print(i)
    if i in predict_data_ids:
        print(i)

1350
1350


In [16]:
num_match = 0
map_dict =  {'A':'zero','B':'one','C':'multiple'}
for i in valid_data:
    pred = pred_results[i['id']]
    if map_dict[pred['prediction']] in i['total_answer']:
        num_match += 1

print(f"accuracy: {num_match/len(valid_data)}")

accuracy: 0.8


## append step num f1

In [4]:
import os
class Args:
    model_name = 't5-large'

In [4]:
#read /apdcephfs/private_zivtang/code/Adaptive-RAG/predictions/test/ircot_qa_flan_t5_xl/total/stepNum.json
import json
with open('/apdcephfs/private_zivtang/code/Adaptive-RAG/predictions/test/ircot_qa_flan_t5_xl/total/stepNum.json', 'r') as f:
    stepNum_dic = json.load(f)
# with open('/apdcephfs/private_zivtang/code/Adaptive-RAG/predictions/dev_500/ircot_qa_flan_t5_xl/total/stepNum.json', 'r') as f:
#     stepNum_dic = json.load(f)

for index in range(len(valid_data)):
    if valid_data[index]['id'] in stepNum_dic:
        valid_data[index]['stepNum'] = stepNum_dic[valid_data[index]['id']]
    else:
        raise ValueError(f"train_data[{index}]['id'] not in stepNum_dic")
    

In [6]:
# #save train_data with indent
with open('/apdcephfs/private_zivtang/code/Adaptive-RAG/MAB/valid.json', 'w') as f:
    json.dump(valid_data, f, indent=4)

In [5]:
valid_data

[{'answer': 'C',
  'dataset_name': 'musique',
  'id': '2hop__20059_236755',
  'question': "What is a notable work from the person who said the term 'intellectual property' operates as a catch-all",
  'total_answer': ['multiple'],
  'stepNum': 3},
 {'answer': 'C',
  'dataset_name': 'musique',
  'id': '4hop1__668178_765799_282674_759393',
  'question': 'What is the seat of the county sharing a border with the county where Don Werner was born?',
  'total_answer': ['multiple'],
  'stepNum': 5},
 {'answer': 'C',
  'dataset_name': 'musique',
  'id': '2hop__388467_110882',
  'question': 'What is the birth date of the By Your Side performer?',
  'total_answer': ['multiple'],
  'stepNum': 2},
 {'answer': 'B',
  'dataset_name': 'musique',
  'id': '3hop1__847420_720378_15538',
  'question': 'When did the state, that contains the county where Elmo is found, reinstate the death penalty?',
  'total_answer': ['multiple', 'one'],
  'stepNum': 3},
 {'answer': 'C',
  'dataset_name': 'musique',
  'id': '